# **「PyTorch入門 5. 自動微分」**

## **Automatic Differentiation with torch.autograd**(通过torch.autograd自动微分)

在训练神经网络时，作为一种学习算法，被用作基本的反向传播(back propagation)

反向传播(back propagation)对于模型中各个参数如权重，损失函数与变量的微分值进行调整

在PyTorch中，通过``torch.autograd``计算梯度

In [ ]:
%matplotlib inline

In [ ]:
import torch

x = torch.ones(5)
# input 输入张量
y = torch.zeros(3)
# output 预期输出

w = torch.randn(5, 3, requires_grad=True)
b = torch.randn(3, requires_grad=True)
# requires_grad=True表示需要计算梯度

z = torch.matmul(x, w)+b
# z = x,w做矩阵乘法 + b

# 计算loss
# 二元交叉熵损失
loss = torch.nn.functional.binary_cross_entropy_with_logits(z, y)
print(loss)

tensor(0.7341, grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)


## **テンソル、関数、計算グラフの関係**(张量，函数，计算图的关系)

<img src="https://pytorch.org/tutorials/_images/comp-graph.png" width=50%>

为了找到最适w b 需要用到loss function

requires_grad=True表示需要计算梯度

【注意】

在构建计算图时，应用于张量的函数实际上是函数类的对象。

这些对象定义了在前向传播过程中如何处理输入。

此外，它们还知道如何在反向传播过程中计算梯度。


梯度将存储在张量的 grad_fn 属性中。

In [ ]:
print('Gradient function for z =',z.grad_fn)
print('Gradient function for loss =', loss.grad_fn)

Gradient function for z = <AddBackward0 object at 0x7d23ee277070>
Gradient function for loss = <BinaryCrossEntropyWithLogitsBackward0 object at 0x7d23ee276230>


## **勾配の計算**(梯度的计算)

为了使神经网络中的各个参数都最适化，当给定输入``x``和输出``y``时，对loss function中的各个参数进行偏微分

$\frac{\partial loss}{\partial w}$ 、$\frac{\partial loss}{\partial b}$

为了求出偏微分值，通过``loss.backward()``，求出``w.grad``和``b.grad``

In [ ]:
loss.backward()
print(w.grad)
print(b.grad)

tensor([[0.2133, 0.3034, 0.3007],
        [0.2133, 0.3034, 0.3007],
        [0.2133, 0.3034, 0.3007],
        [0.2133, 0.3034, 0.3007],
        [0.2133, 0.3034, 0.3007]])
tensor([0.2133, 0.3034, 0.3007])


【注意】

``grad``是计算图中的叶节点(leaf node)也就是说当且仅当requires_grad=True时才会计算

不是所有的函数都可以计算梯度

``backward``只运行一次(出于性能考虑)

如果想进行多次，则需要在 ``backward` 时将 ``retain_graph=True` 作为参数传递

## **勾配計算をしない方法**(不进行梯度计算)

计算梯度时默认选项

当不打算计算梯度时，需确保在 ``torch.no_grad()`` 代码块中包含这些代码

In [ ]:
# 计算梯度
z = torch.matmul(x, w)+b
print(z.requires_grad)

# 不计算梯度
with torch.no_grad():
    z = torch.matmul(x, w)+b
print(z.requires_grad)

True
False


同样可以对张量进行``detch()``来实现不计算梯度

In [ ]:
z = torch.matmul(x, w)+b
z_det = z.detach()
print(z_det.requires_grad)

False


## **計算グラフについて補足**(对计算图的补充)

从理论上来说，``autograd``是一种图形，它以有向无环图(Directed Acyclic Graph)的形式存储张量和对张量的运算，并以函数作为构建模块。

在 DAG 中，每个叶节点(leaf)都是一个输入张量，根(root)是一个输出张量。

利用微分的链式法则，从根(root)追踪到每个叶(leaf)，就能确定每个变量的偏导数值。

在正向传播中，``autograd``同时执行以下处理


*   执行指定操作并获取计算结果的张量
*   更新每个 DAG 运算的梯度函数

<br>
在反向传播中，当DAG根张量执行``.backward()``时，``autograd``执行以下处理



*   计算每个变量的 .grad_fn
*   为每个变量的 .grad 属性分配导数值
*   使用微分链规则计算每个叶子张量的导数值





【注意】

在PyTorch中DAG是动态的(在函数计算处理时按照顺序构建)

每次调用 ``.backward()``，``autograd`` 都会再次创建一个新的图

正是由于这一特性，在模型的前向传播过程中可以使用控制流语句（``if`` 和 ``for`` 语句），从而可以在每次迭代时根据需要改变图形的形状、大小和操作。

## **补充**

大多数的情况中，对于输出标量的loss function，在计算某个变量的梯度

但是，函数的输出不一定是标量，可以是任意的张量

在这种情况下，PyTorch计算Jacobian matrix矩阵，而不是梯度

\begin{split}\begin{align}J=\left(\begin{array}{ccc}
   \frac{\partial y_{1}}{\partial x_{1}} & \cdots & \frac{\partial y_{1}}{\partial x_{n}}\\
   \vdots & \ddots & \vdots\\
   \frac{\partial y_{m}}{\partial x_{1}} & \cdots & \frac{\partial y_{m}}{\partial x_{n}}
   \end{array}\right)\end{align}\end{split}

In [ ]:
inp = torch.eye(5, requires_grad=True)
# 生成一个对角线元素为1，其余元素为0的5*5的矩阵

out = (inp+1).pow(2)
# inp中的每个元素 +1 & 平方

out.backward(torch.ones_like(inp), retain_graph=True)
print("First call\n", inp.grad)
# out相对于inp的梯度


out.backward(torch.ones_like(inp), retain_graph=True)
print("\nSecond call\n", inp.grad)
# 再次计算梯度

inp.grad.zero_()
# 清零梯度
out.backward(torch.ones_like(inp), retain_graph=True)
# 计算清零后的梯度
print("\nCall after zeroing gradients\n", inp.grad)

First call
 tensor([[4., 2., 2., 2., 2.],
        [2., 4., 2., 2., 2.],
        [2., 2., 4., 2., 2.],
        [2., 2., 2., 4., 2.],
        [2., 2., 2., 2., 4.]])

Second call
 tensor([[8., 4., 4., 4., 4.],
        [4., 8., 4., 4., 4.],
        [4., 4., 8., 4., 4.],
        [4., 4., 4., 8., 4.],
        [4., 4., 4., 4., 8.]])

Call after zeroing gradients
 tensor([[4., 2., 2., 2., 2.],
        [2., 4., 2., 2., 2.],
        [2., 2., 4., 2., 2.],
        [2., 2., 2., 4., 2.],
        [2., 2., 2., 2., 4.]])
